# EDA for Big Data Derby Competition

## MAIN

### Creating paths to diferent folders

In [1]:
import os
from pathlib import Path

# Get current work directory of file (Notebook dir)
NOTEBOOK_DIR=Path(os.path.abspath(os.getcwd()))
# Root path of repo
WORK_DIR=NOTEBOOK_DIR.parent.parent
# Path to Datasets
DATA_DIR=WORK_DIR / 'data'

# Noob printing to check if I did this shit right
print("Notebook dir: ", NOTEBOOK_DIR)
print("Repo dir: ", WORK_DIR)
print("Data dir: ", DATA_DIR)

Notebook dir:  c:\Users\Nad\Documents\Programación\Kaggle\big-data-derby-kaggle\notebooks\nad
Repo dir:  c:\Users\Nad\Documents\Programación\Kaggle\big-data-derby-kaggle
Data dir:  c:\Users\Nad\Documents\Programación\Kaggle\big-data-derby-kaggle\data


### Loading csv into dataframes

In [2]:
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns

# horse_data = pd.read_csv(DATA_DIR / 'horse_ids.csv', index_col=0)
df_tracking = pd.read_csv(DATA_DIR / 'ordered_tracking.csv', dtype={
    'race_date': 'string',
    'program_number': 'string'
})
df_tracking['program_number'] = df_tracking['program_number'].str.ljust(3)
df_tracking = df_tracking.groupby(['race_date','race_number','program_number','trakus_index']).min()
df_results = pd.read_csv(DATA_DIR / 'ordered_races.csv').groupby(['race_date','race_number','program_number']).min()


#### Loader functions

- Function output dataframes saved to csv to save time
- Kept commented out just to check how they were made if needed

In [3]:
# def load_data():
#     complete_data_headers = ['track_id','race_date','race_number','program_number','trakus_index','latitude','longitude','distance_id',
#     'course_type','track_condition','run_up_distance','race_type','purse','post_time','weight_carried','jockey','odds','position_at_finish']
#     dtypes = {'race_date': 'string',
#             'program_number': 'string',
#             }
#     complete_data= pd.read_csv(DATA_DIR / 'nyra_2019_complete.csv', names=complete_data_headers, dtype=dtypes)

#     loaded = complete_data.drop(['trakus_index','latitude','longitude','purse','post_time'], axis = 1)
#     loaded['program_number'] = loaded['program_number'].str.ljust(3)
#     loaded = loaded.groupby(['race_date','race_number','program_number']).max()

#     return loaded

In [4]:
## Ordering tracking table by date, number, program and trakus index
## Loading time: ~11 min
## Saved to csv 'ordered_tracking.csv' to save time 

# tracking_df = pd.read_csv(DATA_DIR / 'nyra_tracking_table.csv')
# ordered_tracking = tracking_df.groupby(['race_date','race_number','program_number','trakus_index']).min()
# ordered_tracking.to_csv(DATA_DIR / 'ordered_tracking.csv')


#### Other functions

In [59]:
# Let's def function to apply Haversine formula
# Passing a dataframe with coordinates (lat, lon)
# we get returned a list of distances between the coordinates

# this was modified from a snippet of code extracted from
# https://www.geeksforgeeks.org/program-distance-two-points-earth/ by Aarti_Rathi
def haversine_distance(coords):
    
    # Convert df from degrees to radians.
    rad_coords = np.radians(coords.drop(['speed','distance_covered'],axis=1))

    # Create a shifted version of the coordinates to compare
    shifted_rad_coords = rad_coords.shift(1)

    # And subtract them to get our delta from point a to point b
    sub_coords = shifted_rad_coords.sub(rad_coords)

    dlat, dlon = sub_coords.latitude, sub_coords.longitude

    a = np.sin(dlat / 2)**2 + np.cos(rad_coords.latitude) * np.cos(shifted_rad_coords.latitude) * np.sin(dlon / 2)**2
    
    c = 2 * np.arcsin(np.sqrt(a))

    # Radius of earth in meters.
    r = 6371000
        
    # return the result list
    return (c * r).fillna('0').to_list()

In [70]:
converted_info = False

def add_info(x=500):
    # Add columns to df for new data
    if 'speed' not in df_tracking:
        df_tracking.insert(len(df_tracking.columns), column='speed', value=0.0)
    if 'distance_covered' not in df_tracking:
        df_tracking.insert(len(df_tracking.columns), column='distance_covered', value=0.0)

    # Loop target dataframe index as list
    # TODO modify it to work with a df passed to the function, maybe?
    for i in df_results.head(x).index.to_list():
        distance = haversine_distance(df_tracking.loc[i])

        df_tracking.loc[i, 'speed'] = np.array(distance, dtype='float64') * 4
        df_tracking.loc[i, 'distance_covered'] = np.cumsum(np.array(distance, dtype='float64'))
    
    df_tracking.speed = df_tracking.speed.astype('float64').round(3)
    df_tracking.distance_covered = df_tracking.distance_covered.astype('float64').round(3)

    # Prevent from converting again by running function twice by mistake
    global converted_info

    # Standarize measures to meters
    if converted_info == False:
        df_results.run_up_distance = (df_results.run_up_distance * 0.3048).round(1)
        df_results.distance_id = ((df_results.distance_id / 100) * 201.168).round(1)

        converted_info = True

    return

#### Timings

In [24]:
%timeit haversine_distance(df_tracking.loc['2019-01-01',1,'3  '])

3.44 ms ± 21.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [78]:
%timeit add_info(1)

687 ms ± 4.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [77]:
%timeit add_info(1000)

37.3 s ± 756 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Manipulating dataframes

In [79]:
add_info()

## Snippet testing

In [37]:
for i in df_results.head(500).index.to_list():
    idx = (i[0], i[1], str(i[2]).ljust(3))

    df_results['speed'].loc[idx] = haversine_distance(df_tracking.loc[idx])

In [15]:
df_results.jockey.describe()

count             15081
unique              178
top       Manuel Franco
freq               1289
Name: jockey, dtype: object

In [16]:
df_results.loc[df_results.position_at_finish == 3].jockey.describe()

count              2022
unique               86
top       Manuel Franco
freq                195
Name: jockey, dtype: object

In [17]:
df_results.loc[df_results.position_at_finish == 3].jockey.value_counts()

Manuel Franco       195
Dylan Davis         153
Jose Lezcano        129
Junior Alvarado     124
Irad Ortiz Jr.      121
                   ... 
Thomas Garner         1
Ross Geraghty         1
William McCarthy      1
Nazario Alvarado      1
Wilmer A. Garcia      1
Name: jockey, Length: 86, dtype: int64